In [1]:
import filepath

In [2]:
# from panda3d.core import 
from panda3d.direct import throw_new_frame, init_app_for_gui
from direct.showbase.ShowBase import ShowBase
from panda3d_game.app import ContextShowBase

In [3]:
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QTimer
from PyQt5 import QtCore, QtWidgets
import sys

# from panda3d.core import init_app_for_gui

class ShowBaseQt(ContextShowBase):

    def __init__(self):
        if not hasattr(self, 'isQtInit'):
            ContextShowBase.__init__(self)
            # 这里初始化其他的Panda3D设置
            self.qtApp = None
            self.qtTimer = None
            self.isQtInit = True

    def startQt(self):
        if not QApplication.instance():
            self.qtApp = QApplication([])

        self.spawnQtLoop()

    def spawnQtLoop(self):
        """ 配置定时器让Qt接管主循环 """
        if not self.qtApp:
            self.qtApp = QApplication([])

        init_app_for_gui()

        # 设置一个定时器，每秒调用 60 次 `taskMgr.step()` 更新 Panda3D
        qtFrameRate = 60.0
        self.qtTimer = QTimer()
        self.qtTimer.timeout.connect(self.__qtTimerCallback)
        self.qtTimer.start(int(1000.0 / qtFrameRate))

        # 将主循环交给 Qt 处理
        self.run = self.qtRun
        self.taskMgr.run = self.qtRun

    def __qtTimerCallback(self):
        """ 定时器回调，每次执行 Panda3D 的一个循环 """
        self.taskMgr.step()

    def qtRun(self):
        """ 替换原来的 `run()` 方法，由 Qt 接管主循环 """
        self.qtApp.exec_()



# 使用 ShowBaseQt 作为主类
# base = ShowBaseQt()
# base.startQt()
# base.run()


In [4]:
from direct.showbase.ShowBase import ShowBase
from panda3d.core import WindowProperties
# import tkinter as tk

class PandaInQt(ShowBaseQt):
    def __init__(
        self, 
        # root
    ):
        ShowBase.__init__(self, windowType='none')
        self.isShowBaseInit = True
        ShowBaseQt.__init__(self)
        self.startQt()
        # self.root = root
        # self.frame = tk.Frame(root, width=800, height=600)
        # self.frame.pack(fill=tk.BOTH, expand=tk.YES)
        # self.frame.update()

        # props = WindowProperties()
        # props.set_origin(0, 0)
        # props.set_size(self.frame.winfo_width(), self.frame.winfo_height())
        # props.set_parent_window(self.frame.winfo_id())
        # self.win = self.open_default_window(props=props)
        print("self.qtapp",self.qtApp)
        scene = self.loader.load_model("environment")
        scene.reparent_to(self.render)
        
    def create_window_in_dock(self, window_id):
        # 创建新的窗口并嵌入到 Qt dock 中
        wp = WindowProperties()
        wp.set_parent_window(window_id)  # 设置窗口 ID
        w = self.openDefaultWindow(props=wp)
        # w = self.openWindow(props=wp, type='onscreen',keepCamera=True)  # 创建并打开窗口
        print(w)  # 创建并打开窗口

class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()

        # 设置主窗口标题和大小
        self.setWindowTitle('Panda3D in PyQt5 Dock')
        self.setGeometry(100, 100, 800, 600)

        # 创建一个可停靠的窗口 (dock)
        self.dock = QtWidgets.QDockWidget('Panda3D View', self)
        self.addDockWidget(QtCore.Qt.RightDockWidgetArea, self.dock)

        # 创建一个 QWidget 作为 dock 的内容
        self.panda3d_container = QtWidgets.QWidget(self.dock)
        self.dock.setWidget(self.panda3d_container)

        # 启动 Panda3D
        # self.start_panda3d()

    # def start_panda3d(self):
    #     # 启动 Panda3D 的线程
    #     self.panda3d_thread = threading.Thread(target=self.run_panda3d, daemon=True)
    #     self.panda3d_thread.start()

    def run_panda3d(self):
        # 初始化没有窗口的 Panda3D
        self.panda_app = Panda3DApp()

    def create_panda3d_window(self):
        # 获取 QWidget 的窗口句柄 ID
        window_id = self.panda3d_container.winId()
        # 在 Panda3D 中创建新窗口，并嵌入 Qt 的 dock 中
        self.panda_app.create_window_in_dock(window_id)
# # root = tk.Tk()
# app = PandaInTk(root)
# root.mainloop()



In [ ]:
if __name__ == '__main__':
    import builtins
    import traceback
    try:
        app = QtWidgets.QApplication(sys.argv)

        # 创建并显示 PyQt 主窗口
        main_window = MainWindow()
        main_window.show()
        print("main window",main_window)
        
        window_id = main_window.panda3d_container.winId()
        print(window_id, int(window_id))
        try:
            with PandaInQt() as app_:
                app_.create_window_in_dock(int(window_id))
                # sys.exit(app_.run)
                # app_.run()
        except Exception as e_:
            raise e_
        # 延迟创建 Panda3D 窗口
        # QtCore.QTimer.singleShot(1000, main_window.create_panda3d_window)
        finally:
            sys.exit(app.exec_())
    except Exception as e:
        print(e)
        print(traceback.format_exc())
    finally:
        if hasattr(builtins, 'base'):
            builtins.base.destroy()    

main window <__main__.MainWindow object at 0x7f852fc9a050>
<sip.voidptr object at 0x7f852faeb150> 65011729
init ContextShowBase
self.qtapp <PyQt5.QtWidgets.QApplication object at 0x7f8529f84c10>
---<__main__.PandaInQt object at 0x7f852fc1e010> destroy at 2024-09-15 09:39:49.030035---
---<__main__.PandaInQt object at 0x7f852fc1e010> destroyed at 2024-09-15 09:39:49.030472, exit---


:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:ShowBase(warning): Unable to open 'none' window.


In [1]:
from PyQt5 import QtWidgets
from PyQt5.QtCore import QTimer
from panda3d.core import WindowProperties
from direct.showbase.ShowBase import ShowBase
import sys

class MyApp(ShowBase):
    def __init__(self, qt_window):
        ShowBase.__init__(self, windowType='none')  # 不启动默认窗口
        self.qt_window = qt_window
    def cw(self):
        # 配置 Qt 窗口
        window_id = int(self.qt_window.winId())
        props = WindowProperties()
        props.set_parent_window(window_id)
        self.openDefaultWindow(props=props)

# PyQt5 应用
class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Panda3D in PyQt5')
        self.setGeometry(100, 100, 800, 600)
        self.panda_widget = QtWidgets.QWidget(self)
        self.setCentralWidget(self.panda_widget)
        self.show()

# 启动 Qt 应用并嵌入 Panda3D
app = QtWidgets.QApplication(sys.argv)
window = MainWindow()

panda_app = MyApp(window.panda_widget)
# 使用定时器来协调 Panda3D 的主循环
timer = QTimer()
panda_app.taskMgr.step()
timer.timeout.connect(panda_app.taskMgr.step)

timer.start(16)  # 约 60 FPS
panda_app.cw()
  # 确保 Panda3D 渲染开始
sys.exit(app.exec_())


:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:ShowBase(warning): Unable to open 'none' window.


Exception: Could not open window.

In [6]:
# -*- coding: utf-8-*-
"""
Module : Panda3DWorld
Author : Saifeddine ALOUI
Description :
    Inherit this object to create your custom world
"""

# PyQt imports
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

# Panda imports
from panda3d.core import *
# from panda3d.core import loadPrcFileData
# loadPrcFileData("", "window-type none") # Set Panda to draw its main window in an offscreen buffer
from direct.showbase.DirectObject import DirectObject
from panda3d.core import GraphicsOutput, Texture, ConfigVariableManager, WindowProperties

# Set up Panda environment
from direct.showbase.ShowBase import ShowBase
import platform

from panda3d_game.app import ContextShowBase 
from QPanda3D.QMouseWatcherNode import QMouseWatcherNode
from QPanda3D import Panda3DWorld



In [11]:
# -*- coding: utf-8-*-
"""
Module : Panda3DWorld
Author : Saifeddine ALOUI
Description :
    Inherit this object to create your custom world
"""

# PyQt imports
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

# Panda imports
from panda3d.core import *
# from panda3d.core import loadPrcFileData
# loadPrcFileData("", "window-type none") # Set Panda to draw its main window in an offscreen buffer
from direct.showbase.DirectObject import DirectObject
from panda3d.core import GraphicsOutput, Texture, ConfigVariableManager, WindowProperties

# Set up Panda environment
from direct.showbase.ShowBase import ShowBase
import platform

# Local imports
from QPanda3D.QMouseWatcherNode import QMouseWatcherNode

__all__ = ["Panda3DWorld"]


class Panda3DWorld_(ShowBase):
    """
    Panda3DWorld : A class to handle all panda3D world manipulation
    """

    def __init__(self, width=800, height=600, is_fullscreen=False, size=1.0, clear_color=LVecBase4f(0.1, 0.1, 0.1, 1),
                 name="qpanda3D"):

        sort = -100
        self.parent = None
        # self.width = width
        # self.height = height

        loadPrcFileData("", "win-size {} {}".format(width, height))

        if (is_fullscreen):
            loadPrcFileData("", "fullscreen #t")
        else:
            loadPrcFileData("", "window-type offscreen")  # Set Panda to draw its main window in an offscreen buffer

        ShowBase.__init__(self)
        self.screenTexture = Texture()
        self.screenTexture.setMinfilter(Texture.FTLinear)
        self.screenTexture.setFormat(Texture.FRgba32)
        self.screenTexture.set_wrap_u(Texture.WM_clamp)
        self.screenTexture.set_wrap_v(Texture.WM_clamp)

        buff_size_x = int(self.win.get_x_size() * size)
        buff_size_y = int(self.win.get_y_size() * size)
        winprops = WindowProperties()
        winprops.set_size(buff_size_x, buff_size_y)

        props = FrameBufferProperties()
        props.set_rgb_color(True)
        props.set_rgba_bits(8, 8, 8, 8)
        props.set_depth_bits(8)

        self.buff = self.graphicsEngine.make_output(
            self.pipe, name, sort,
            props, winprops,
            GraphicsPipe.BF_resizeable,
            self.win.get_gsg(), self.win)

        self.buff.addRenderTexture(self.screenTexture, GraphicsOutput.RTMCopyRam)
        self.buff.set_sort(sort)
        self.cam = self.makeCamera(self.buff)
        self.camNode = self.cam.node()
        self.camLens = self.camNode.get_lens()

        if clear_color is None:
            self.buff.set_clear_active(GraphicsOutput.RTPColor, False)
        else:
            self.buff.set_clear_color(clear_color)
            self.buff.set_clear_active(GraphicsOutput.RTPColor, True)

        self.disableMouse()

    def set_parent(self, parent: QWidget):
        self.parent = parent
        self.mouseWatcherNode = QMouseWatcherNode(parent)

    def getAspectRatio(self, win = None):
        if win is None and self.parent is not None:
            return float(self.parent.width()) / float(self.parent.height())
        else:
            return super().getAspectRatio(win)


In [3]:
# -*- coding: utf-8-*-
"""
Module : Panda3DWorld
Author : Saifeddine ALOUI
Description :
    Inherit this object to create your custom world
"""

# PyQt imports
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

# Panda imports
from panda3d.core import *
# from panda3d.core import loadPrcFileData
# loadPrcFileData("", "window-type none") # Set Panda to draw its main window in an offscreen buffer
from direct.showbase.DirectObject import DirectObject
from panda3d.core import GraphicsOutput, Texture, ConfigVariableManager, WindowProperties

# Set up Panda environment
from direct.showbase.ShowBase import ShowBase
import platform

# Local imports
from QPanda3D.QMouseWatcherNode import QMouseWatcherNode

__all__ = ["Panda3DWorld"]


class QShowBase(ContextShowBase):
    """
    edited from Panda3DWorld
    without disabling mouse
    """

    def __init__(
        self,  
        size=1.0, 
        clear_color=LVecBase4f(0.1, 0.1, 0.1, 1),
        name="qpanda3D"
    ):
        sort = -100
        self.parent = None

        ShowBase.__init__(self)
        self.screenTexture = Texture()
        self.screenTexture.setMinfilter(Texture.FTLinear)
        self.screenTexture.setFormat(Texture.FRgba32)
        self.screenTexture.set_wrap_u(Texture.WM_clamp)
        self.screenTexture.set_wrap_v(Texture.WM_clamp)

        buff_size_x = int(self.win.get_x_size() * size)
        buff_size_y = int(self.win.get_y_size() * size)
        winprops = WindowProperties()
        winprops.set_size(buff_size_x, buff_size_y)

        props = FrameBufferProperties()
        props.set_rgb_color(True)
        props.set_rgba_bits(8, 8, 8, 8)
        props.set_depth_bits(8)

        self.buff = self.graphicsEngine.make_output(
            self.pipe, name, sort,
            props, winprops,
            GraphicsPipe.BF_resizeable,
            self.win.get_gsg(), self.win)

        self.buff.addRenderTexture(self.screenTexture, GraphicsOutput.RTMCopyRam)
        self.buff.set_sort(sort)
        self.cam = self.makeCamera(self.buff)
        self.camNode = self.cam.node()
        self.camLens = self.camNode.get_lens()

        if clear_color is None:
            self.buff.set_clear_active(GraphicsOutput.RTPColor, False)
        else:
            self.buff.set_clear_color(clear_color)
            self.buff.set_clear_active(GraphicsOutput.RTPColor, True)


    def set_parent(self, parent: QWidget):
        self.parent = parent
        self.mouseWatcherNode = QMouseWatcherNode(parent)

    def getAspectRatio(self, win = None):
        if win is None and self.parent is not None:
            return float(self.parent.width()) / float(self.parent.height())
        else:
            return super().getAspectRatio(win)


In [4]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextEdit, QVBoxLayout, QWidget, QPushButton, QDockWidget
from PyQt5.QtCore import Qt

class CommandLineWidget(QWidget):
    def __init__(self):
        super().__init__()
        
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)

        # Create a text edit for command input
        self.command_input = QTextEdit()
        self.command_input.setPlaceholderText("Enter commands here...")
        self.command_input.setMaximumHeight(100)
        self.layout.addWidget(self.command_input)

        # Create a button to execute commands
        self.execute_button = QPushButton("Execute")
        self.execute_button.clicked.connect(self.execute_command)
        self.layout.addWidget(self.execute_button)

    def execute_command(self):
        command = self.command_input.toPlainText().strip()
        if command:
            # You would normally handle the command here
            print(f"Command executed: {command}")
            self.command_input.clear()

class LogWidget(QWidget):
    def __init__(self):
        super().__init__()

        self.layout = QVBoxLayout()
        self.setLayout(self.layout)

        # Create a text edit for displaying logs
        self.log_display = QTextEdit()
        self.log_display.setReadOnly(True)
        self.layout.addWidget(self.log_display)

    def add_log(self, message):
        self.log_display.append(message)

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Main Window")
        self.setGeometry(100, 100, 800, 600)

#         # Create Command Line Dock Widget
#         self.command_line_widget = CommandLineWidget()
#         self.command_line_dock = QDockWidget("Command Line", self)
#         self.command_line_dock.setWidget(self.command_line_widget)
#         self.addDockWidget(Qt.LeftDockWidgetArea, self.command_line_dock)

#         # Create Log Dock Widget
#         self.log_widget = LogWidget()
#         self.log_dock = QDockWidget("Log", self)
#         self.log_dock.setWidget(self.log_widget)
#         self.addDockWidget(Qt.RightDockWidgetArea, self.log_dock)

        self.setWindowTitle('Panda3D in PyQt5 Dock')
        self.setGeometry(100, 100, 800, 600)

        # 创建一个可停靠的窗口 (dock)
        self.dock = QtWidgets.QDockWidget('Panda3D View', self)
        self.addDockWidget(QtCore.Qt.RightDockWidgetArea, self.dock)

        # 创建一个 QWidget 作为 dock 的内容
        self.panda3d_container = QtWidgets.QWidget(self.dock)
        self.dock.setWidget(self.panda3d_container)

    def get_log_widget(self):
        return self.log_widget

In [5]:
class MyWorld(QShowBase):
    def __init__(self):
        QShowBase.__init__(self)
        # from this point, act as if you are defining a classic panda3D environment
        self.cam.setPos(0, -28, 6)
        scene = self.loader.load_model("environment")
        scene.reparent_to(self.render)
        

In [6]:
if __name__ == "__main__":
    is_fullscreen=False
    width=800
    height=600,

    loadPrcFileData("", "win-size {} {}".format(width, height))

    if (is_fullscreen):
        loadPrcFileData("", "fullscreen #t")
    else:
        loadPrcFileData("", "window-type offscreen")  # Set Panda to draw its main window in an offscreen buffer
        
    app = QApplication(sys.argv)
    appw=QMainWindow()
    appw.setGeometry(50, 50, 800, 600)
    papp = QShowBase()
    pandaWidget = QPanda3DWidget(papp)
    appw.setCentralWidget(pandaWidget)
    appw.show()
    sys.exit(app.exec_())

:prc(warning): Invalid integer value for ConfigVariable win-size: (600,)
Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
Assertion failed: _pbuffer == None at line 53 of panda/src/glxdisplay/glxGraphicsBuffer.cxx


AssertionError: _pbuffer == None at line 53 of panda/src/glxdisplay/glxGraphicsBuffer.cxx

In [4]:
if __name__ == "__main__":
    is_fullscreen=False
    width=800
    height=600,

    loadPrcFileData("", "win-size {} {}".format(width, height))

    if (is_fullscreen):
        loadPrcFileData("", "fullscreen #t")
    else:
        loadPrcFileData("", "window-type offscreen")  # Set Panda to draw its main window in an offscreen buffer
        
    world = MyWorld()
    
    app = QApplication(sys.argv)
    appw=QMainWindow()
    appw.setGeometry(50, 50, 800, 600)

    pandaWidget = QPanda3DWidget(world)
    appw.setCentralWidget(pandaWidget)
    appw.show()
    
    sys.exit(app.exec_())

NameError: name 'MyWorld' is not defined

In [ ]:
from QPanda3D.Panda3DWorld import Panda3DWorld
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextEdit, QVBoxLayout, QWidget, QPushButton, QDockWidget
from PyQt5.QtCore import Qt
class MyWorld(QShowBase):
    def __init__(self):
        Panda3DWorld.__init__(self)
        # from this point, act as if you are defining a classic panda3D environment
        self.cam.setPos(0, -28, 6)
        self.testModel = loader.loadModel('panda')
        self.testModel.reparentTo(render)
from QPanda3D.QPanda3DWidget import QPanda3DWidget
if __name__ == "__main__":
    is_fullscreen=False
    width=800
    height=600,

    loadPrcFileData("", "win-size {} {}".format(width, height))

    if (is_fullscreen):
        loadPrcFileData("", "fullscreen #t")
    else:
        
    
        loadPrcFileData("", "window-type offscreen")  # Set Panda to draw its main window in an offscreen buffer
    app = QApplication(sys.argv)
    
    with MyWorld() as world:
        # world = MyWorld()

        appw=QMainWindow()
        appw.setGeometry(50, 50, 800, 600)

        pandaWidget = QPanda3DWidget(world)
        appw.setCentralWidget(pandaWidget)
        appw.show()

    sys.exit(app.exec_())

In [6]:
world.destroy()